In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("013. K-means").getOrCreate()

In [2]:
datos=spark.read.format("libsvm").load("data/sample_kmeans_data.txt")
datos.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [3]:
datos.show(truncate=False)

+-----+-------------------------+
|label|features                 |
+-----+-------------------------+
|0.0  |(3,[],[])                |
|1.0  |(3,[0,1,2],[0.1,0.1,0.1])|
|2.0  |(3,[0,1,2],[0.2,0.2,0.2])|
|3.0  |(3,[0,1,2],[9.0,9.0,9.0])|
|4.0  |(3,[0,1,2],[9.1,9.1,9.1])|
|5.0  |(3,[0,1,2],[9.2,9.2,9.2])|
+-----+-------------------------+



In [4]:
from pyspark.ml.clustering import KMeans

In [13]:
km=KMeans(k=2)

In [6]:
modelo=km.fit(datos)

In [7]:
modelo.summary.predictions.show(truncate=False)

+-----+-------------------------+----------+
|label|features                 |prediction|
+-----+-------------------------+----------+
|0.0  |(3,[],[])                |0         |
|1.0  |(3,[0,1,2],[0.1,0.1,0.1])|0         |
|2.0  |(3,[0,1,2],[0.2,0.2,0.2])|0         |
|3.0  |(3,[0,1,2],[9.0,9.0,9.0])|1         |
|4.0  |(3,[0,1,2],[9.1,9.1,9.1])|1         |
|5.0  |(3,[0,1,2],[9.2,9.2,9.2])|1         |
+-----+-------------------------+----------+



In [8]:
modelo.computeCost(datos)

0.11999999999994547

In [9]:
modelo.clusterCenters()

[array([ 0.1,  0.1,  0.1]), array([ 9.1,  9.1,  9.1])]

In [10]:
resultado=modelo.transform(datos)# aplicamos el modelo a datos nuevosÇ

In [11]:
resultado.show()

+-----+--------------------+----------+
|label|            features|prediction|
+-----+--------------------+----------+
|  0.0|           (3,[],[])|         0|
|  1.0|(3,[0,1,2],[0.1,0...|         0|
|  2.0|(3,[0,1,2],[0.2,0...|         0|
|  3.0|(3,[0,1,2],[9.0,9...|         1|
|  4.0|(3,[0,1,2],[9.1,9...|         1|
|  5.0|(3,[0,1,2],[9.2,9...|         1|
+-----+--------------------+----------+



In [14]:
km3=KMeans(k=3,seed=1)

In [15]:
modelo3=km3.fit(datos)

In [17]:
modelo3.summary.predictions.show()

+-----+--------------------+----------+
|label|            features|prediction|
+-----+--------------------+----------+
|  0.0|           (3,[],[])|         1|
|  1.0|(3,[0,1,2],[0.1,0...|         1|
|  2.0|(3,[0,1,2],[0.2,0...|         2|
|  3.0|(3,[0,1,2],[9.0,9...|         0|
|  4.0|(3,[0,1,2],[9.1,9...|         0|
|  5.0|(3,[0,1,2],[9.2,9...|         0|
+-----+--------------------+----------+



In [20]:
modelo3.clusterCenters()

[array([ 9.1,  9.1,  9.1]),
 array([ 0.05,  0.05,  0.05]),
 array([ 0.2,  0.2,  0.2])]

In [21]:
modelo3.computeCost(datos)

0.07499999999994544

In [22]:
datosNuevos=spark.createDataFrame([(2,3,5),(2,9,1)])

In [23]:
datosNuevos.show()

+---+---+---+
| _1| _2| _3|
+---+---+---+
|  2|  3|  5|
|  2|  9|  1|
+---+---+---+



In [24]:
from pyspark.ml.feature import VectorAssembler

In [25]:
constructor=VectorAssembler(inputCols=["_1","_2","_3"],
                           outputCol="features")

In [26]:
datosFinales=constructor.transform(datosNuevos)

In [27]:
datosFinales.show()

+---+---+---+-------------+
| _1| _2| _3|     features|
+---+---+---+-------------+
|  2|  3|  5|[2.0,3.0,5.0]|
|  2|  9|  1|[2.0,9.0,1.0]|
+---+---+---+-------------+



In [28]:
resultado=modelo.transform(datosFinales)

In [29]:
resultado.show()

+---+---+---+-------------+----------+
| _1| _2| _3|     features|prediction|
+---+---+---+-------------+----------+
|  2|  3|  5|[2.0,3.0,5.0]|         0|
|  2|  9|  1|[2.0,9.0,1.0]|         0|
+---+---+---+-------------+----------+



In [30]:
resultado3=modelo3.transform(datosFinales)
resultado3.show()

+---+---+---+-------------+----------+
| _1| _2| _3|     features|prediction|
+---+---+---+-------------+----------+
|  2|  3|  5|[2.0,3.0,5.0]|         2|
|  2|  9|  1|[2.0,9.0,1.0]|         2|
+---+---+---+-------------+----------+

